# **BOL Product Dimension Validator**

Purpose of this notebook is to express solutions and validations of the shipment costs and their tariff sizes for multi platform e-commerce businesses.

**WORK CASE:**
Consider you are selling products in two well known e-commerce platforms Amazon and Bol. Both platforms have different tariff for several products according to their product and package sizes for shipments. To be able to validate that your business is getting cut correct tariff size and products are evaluated with correct sizes in the platforms. This approach is a basic automation approach with given inputs of the invoices and the products, following months and validation for financial departments of the businesses can be performed with couple clicks instead of huge sized excel files and manual validation methods. Instead of days long workload can be performed each month with couple clicks ahead! 

**Requirements and Packages** 

In [3]:
import os
import pandas as pd 
import numpy as np 

**Data**

Consider your business have a report for the collection of products with their dimensions for variables Length, Width, Height and Unique identifier code.

In [4]:
dim_df = pd.read_excel('product_dimensions.xlsx')
dim_df.head()

,Productidentificatie,Length,Width,Height
0,8719992661473,19.6,15.6,11.6
1,8720828495247,61.7,27.8,20.6
2,8720618229236,31.8,10.5,10.1
3,8720618229908,47.0,14.2,8.8
4,8720828495223,12.7,12.7,6.5


Further the collection of invoice reports provided from platforms with respect to tarif groups costs, location, unique identifiers of the products.

In [6]:
aug = pd.read_excel('Bol Dimensions Aug.xlsx')
sep = pd.read_excel('Bol Dimensions Sept.xlsx')


df = pd.concat([aug, sep], ignore_index=True)
df.head()

,Type,Type productidentificatie,Productidentificatie,Artikelomschrijving,Datum,Bestelnummer,Aantal,Tarief-\ngroep,Tarief,Bedrag,BTW %,Btw-bedrag,Bedrag\n(incl. BTW),Land van verzending,Reden,Opmerking
0,Pick&pack kosten,EAN,8721001270170,Piercing Kussen - Kussen Met Gat Voor Oor - Do...,2024-08-01,4143871367,1,M,2.19,2.19,21,0.4599,2.6499,NL,NaN,NaN
1,Pick&pack kosten,EAN,8720618229939,LittleLaughs Verschoningsmatje Voor Onderweg -...,2024-08-01,4144919007,1,S,2.07,2.07,21,0.4347,2.5047,NL,NaN,NaN
2,Pick&pack kosten,EAN,8720618229946,LP Opbergsysteem | Vintage Design | LP Houder ...,2024-08-01,4150611944,1,XS,1.65,1.65,21,0.3465,1.9965,BE,NaN,NaN
3,Pick&pack kosten,EAN,8720618229687,Mijlpaaldeken In Nederlands - Milestone Deken ...,2024-08-01,4141622027,1,S,2.07,2.07,21,0.4347,2.5047,NL,NaN,NaN
4,Pick&pack kosten,EAN,8720618229731,Mosstok voor Planten - Verlengbaar tot 140cm -...,2024-08-01,4148574231,1,M,2.19,2.19,21,0.4599,2.6499,NL,NaN,NaN


**Bol.com Dimension Tariff Validator Function**

Function below performs operation over two dataframes, first the one user provides with their actual packaging sizes for the products and second one is the invoice report from the platform. Function first evaluates your product sizes into tariff size provided from the Bol.com labels them accordingly and stores. Once this step is performed goes through the invoice reports and runs through all entries to compare products with respect to their identifier codes and validates if the size is labeled correct or not. Following steps for the output will be if the product tariff size is labeled wrong extracts the entry in the invoice report and stores in a list, another output is the correct tarif size for the products. With this application both businesses can find the mistakes occured in the system and easily extract which product labeled wrong and fix the price gaps in the invoices to ensure a safe and legit transactions are ongoing. 

In [13]:
def bol_dim_validate(df,df2):

    df2['tarief_size'] = ''
    wrong_dimensions = []

    for i in range(0,len(df2['Productidentificatie'])):
        if (df2['Length'].iloc[i] < 23.5) & (df2['Width'].iloc[i] < 16.5) & (df2['Height'].iloc[i] < 3):
            df2.loc[i,'tarief_size'] = '3XS'
        elif (df2['Length'].iloc[i] < 37.5) & (df2['Width'].iloc[i] < 26) & (df2['Height'].iloc[i] < 3):
            df2.loc[i,'tarief_size'] = 'XXS'
        elif (df2['Length'].iloc[i] < 37.5) & (df2['Width'].iloc[i] < 26) & (df2['Height'].iloc[i] < 5):
            df2.loc[i,'tarief_size'] = 'XS'
        elif (df2['Length'].iloc[i] < 45) & (df2['Width'].iloc[i] < 30) & (df2['Height'].iloc[i] < 8):
            df2.loc[i,'tarief_size'] = 'S'
        elif (df2['Length'].iloc[i] < 55) & (df2['Width'].iloc[i] < 35) & (df2['Height'].iloc[i] < 20):
            df2.loc[i,'tarief_size'] = 'M'
        elif (df2['Length'].iloc[i] < 72) & (df2['Width'].iloc[i] < 50) & (df2['Height'].iloc[i] < 41):
            df2.loc[i,'tarief_size'] = 'L'
        else:
            df2.loc[i,'tarief_size'] = 'XL'

    for id in df['Productidentificatie'].unique():
        id_df = df[df['Productidentificatie'] == id]

        size_df = df2[df2['Productidentificatie'] == id]

        # Convert unique values to sets for comparison
        tariefgroep_unique = id_df['Tarief-\ngroep'].unique()
        tariefsize_unique = size_df['tarief_size'].unique()

        if (len(tariefgroep_unique) == len(tariefsize_unique)) and (tariefgroep_unique != tariefsize_unique):
            wrong_dimensions.append(id_df)
        else:
            for i in range(0,len(tariefgroep_unique)):
                if tariefgroep_unique[i] != tariefsize_unique[0]:
                    wrong_dimensions.append(id_df[id_df['Tarief-\ngroep'] == tariefgroep_unique[i]])

    wrong_dimensions

    wrong_df = pd.concat(wrong_dimensions, ignore_index=True)

    size_catalogue = df2.copy()

    trimmed_wrong = wrong_df[['Productidentificatie','Tarief-\ngroep']]
    unique_products_tarief = trimmed_wrong[['Productidentificatie', 'Tarief-\ngroep']].drop_duplicates(subset='Productidentificatie').reset_index(drop = True)

    comparison_merged =pd.merge(size_catalogue, unique_products_tarief, on= "Productidentificatie", how= "inner")
    comparison_merged.rename(columns={'Tarief-\ngroep': 'bol.com_tarief'}, inplace=True)


    return wrong_df, size_catalogue, comparison_merged


An example case: 

In [14]:
# Uploading the product
dim_df = pd.read_excel('product_dimensions.xlsx')
dim_df.head()

,Productidentificatie,Length,Width,Height
0,8719992661473,19.6,15.6,11.6
1,8720828495247,61.7,27.8,20.6
2,8720618229236,31.8,10.5,10.1
3,8720618229908,47.0,14.2,8.8
4,8720828495223,12.7,12.7,6.5


In [15]:
# Concating and storing the invoice reports into a dataframe. 
aug = pd.read_excel('Bol Dimensions Aug.xlsx')
sep = pd.read_excel('Bol Dimensions Sept.xlsx')

invoices_df = pd.concat([aug, sep], ignore_index=True)
invoices_df.head()

,Type,Type productidentificatie,Productidentificatie,Artikelomschrijving,Datum,Bestelnummer,Aantal,Tarief-\ngroep,Tarief,Bedrag,BTW %,Btw-bedrag,Bedrag\n(incl. BTW),Land van verzending,Reden,Opmerking
0,Pick&pack kosten,EAN,8721001270170,Piercing Kussen - Kussen Met Gat Voor Oor - Do...,2024-08-01,4143871367,1,M,2.19,2.19,21,0.4599,2.6499,NL,NaN,NaN
1,Pick&pack kosten,EAN,8720618229939,LittleLaughs Verschoningsmatje Voor Onderweg -...,2024-08-01,4144919007,1,S,2.07,2.07,21,0.4347,2.5047,NL,NaN,NaN
2,Pick&pack kosten,EAN,8720618229946,LP Opbergsysteem | Vintage Design | LP Houder ...,2024-08-01,4150611944,1,XS,1.65,1.65,21,0.3465,1.9965,BE,NaN,NaN
3,Pick&pack kosten,EAN,8720618229687,Mijlpaaldeken In Nederlands - Milestone Deken ...,2024-08-01,4141622027,1,S,2.07,2.07,21,0.4347,2.5047,NL,NaN,NaN
4,Pick&pack kosten,EAN,8720618229731,Mosstok voor Planten - Verlengbaar tot 140cm -...,2024-08-01,4148574231,1,M,2.19,2.19,21,0.4599,2.6499,NL,NaN,NaN


In [16]:
# Running the function defined 
wrong_dimensions, correct_sizes, tarif_sizes_comparison = bol_dim_validate(invoices_df,dim_df)

Function consists three different outputs for different first one is the wrong invoice table entries with in shape of the voice report, another table to update correct sizes of the dimension inputs from business' products itself and the last one is comparison of which unique identifier products conflict with the wrong sized products from bol.com 

In [17]:
wrong_dimensions

,Type,Type productidentificatie,Productidentificatie,Artikelomschrijving,Datum,Bestelnummer,Aantal,Tarief-\ngroep,Tarief,Bedrag,BTW %,Btw-bedrag,Bedrag\n(incl. BTW),Land van verzending,Reden,Opmerking
0,Pick&pack kosten,EAN,8721001270170,Piercing Kussen - Kussen Met Gat Voor Oor - Do...,2024-08-01,4143871367,1,M,2.19,2.19,21,0.4599,2.6499,NL,NaN,NaN
1,Pick&pack kosten,EAN,8721001270170,Piercing Kussen - Kussen Met Gat Voor Oor - Do...,2024-08-01,4135954956,1,M,2.19,2.19,21,0.4599,2.6499,NL,NaN,NaN
2,Pick&pack kosten,EAN,8721001270170,Piercing Kussen - Kussen Met Gat Voor Oor - Do...,2024-08-01,4145597012,1,M,2.19,2.19,21,0.4599,2.6499,NL,NaN,NaN
3,Pick&pack kosten,EAN,8721001270170,Piercing Kussen - Kussen Met Gat Voor Oor - Do...,2024-08-02,4147065697,1,M,2.19,2.19,21,0.4599,2.6499,NL,NaN,NaN
4,Pick&pack kosten,EAN,8721001270170,Piercing Kussen - Kussen Met Gat Voor Oor - Do...,2024-08-03,4120184643,1,M,2.19,2.19,21,0.4599,2.6499,NL,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
728,Pick&pack kosten,EAN,8721001271733,Verzwaringsknuffel - Weighted Stuffed Animal -...,2024-09-30,4171580211,1,M,2.19,2.19,21,0.4599,2.6499,BE,NaN,NaN
729,Pick&pack kosten,EAN,8721001271634,Human Dog Bed - Groot Zitzakbed voor Volwassen...,2024-09-10,4166487204,1,XL,4.48,4.48,21,0.9408,5.4208,NL,NaN,NaN
730,Pick&pack kosten,EAN,8721001271634,Human Dog Bed - Groot Zitzakbed voor Volwassen...,2024-09-17,4158356746,1,XL,4.48,4.48,21,0.9408,5.4208,BE,NaN,NaN
731,Pick&pack kosten,EAN,8721001271634,Human Dog Bed - Groot Zitzakbed voor Volwassen...,2024-09-21,4154227905,1,XL,4.48,4.48,21,0.9408,5.4208,BE,NaN,NaN


In [18]:
correct_sizes

,Productidentificatie,Length,Width,Height,tarief_size
0,8719992661473,19.6,15.6,11.6,M
1,8720828495247,61.7,27.8,20.6,L
2,8720618229236,31.8,10.5,10.1,M
3,8720618229908,47.0,14.2,8.8,M
4,8720828495223,12.7,12.7,6.5,S
...,...,...,...,...,...
60,8721001271689,55.0,60.0,55.0,XL
61,8721001271672,59.0,59.0,5.5,XL
62,8721001271764,37.3,31.8,3.3,M
63,8721001271610,38.0,40.0,47.0,XL


In [19]:
tarif_sizes_comparison

,Productidentificatie,Length,Width,Height,tarief_size,bol.com_tarief
0,8720828495223,12.7,12.7,6.5,S,L
1,8720618229687,22.9,22.1,8.0,M,S
2,8721001270248,42.8,33.8,14.4,M,L
3,8721001270170,32.8,27.8,5.1,S,M
4,8719992661176,40.0,30.0,5.0,M,S
5,8719992661107,29.9,9.2,5.0,S,XS
6,8720618229915,30.0,24.0,3.0,XS,XXS
7,8719992661824,26.8,21.3,5.6,S,XXS
8,8719992661220,31.0,23.8,2.9,XXS,3XS
9,8721001271382,46.0,37.0,37.0,L,XL


In [ ]:
# tarif_sizes_comparison.to_excel('comparison_sizes.xlsx',index = False)
# wrong_dimensions.to_excel('wrong_dimensions.xlsx',index = False)
# correct_sizes.to_excel('correct_sizes.xlsx', index=False)